<a href="https://colab.research.google.com/github/sjut/DPO_Materials/blob/master/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Извлечение информации правлами
##Продолжение FOOD
Для извлечения структурированной информации из неструткурированных текстов существуют специальные системы, основанные на правилах.\
Обычно система состоит из:
правила + гезетёры (или газетиры gazetteer)\
*газетеры* - специальные списки - например, списики названий всех организаций, которые должна извлекать система; списки продуктов для извлечения ингредиентов из рецептов и т.п.\
Формально система правил может быть реализована как система конченых преобразователей (Gate), либо как система контекстно-свободных грамматик

[Yargy](https://yargy.readthedocs.io/ru/latest/) — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов (газетерами).\

Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории [Natasha](https://github.com/natasha/natasha).


In [1]:
#устанавливаем yargy - Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. 
!pip install yargy
!pip install ipymarkup #Collection of NLP visualizations for NER and syntax tree markup.

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 
     |████████████████████████████████| 7.1MB 6.1MB/s 


In [4]:
#клонируем папку гитхабе yargy - пример про извлечение данных из рецептов приготовления еды 04_food
%cd /content
!git clone https://github.com/natasha/yargy-examples.git
%cd yargy-examples/04_food

/content
Cloning into 'yargy-examples'...
remote: Enumerating objects: 2087, done.
remote: Total 2087 (delta 0), reused 0 (delta 0), pack-reused 2087
Receiving objects: 100% (2087/2087), 6.25 MiB | 20.79 MiB/s, done.
Resolving deltas: 100% (118/118), done.
/content/yargy-examples/04_food


In [0]:
# Модуль glob находит все пути, совпадающие с заданным шаблоном в соответствии с правилами, используемыми оболочкой Unix.
from glob import glob

# стандартизуем тире,  заменяем (убираем) спецсиволы (неразрывные пробелы, мягкие переносы) и т.п.  

def make_translation(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation(
    '‑–—−',
    '----'
)


def preprocess(text):
    text = text.replace('\xa0', ' ')
    text = text.replace('\xad', '')
    text = text.translate(DASHES_TRANSLATION)
    return text

#считываем тексты из папки с текстами (файлы с расширением .txt) и кладем их в список
texts = []
for path in sorted(glob('texts/*.txt')):
    with open(path) as file:
        text = file.read()
        text = preprocess(text)
        texts.append(text)

In [8]:
#проверяем, что записалось в список, сколько текстов считалось
print(texts[0])
len(texts)

Салат с ананасами и курицей 🍍

Ингредиенты:

● Филе куриной грудки 1 шт.
● Шампиньоны 300 г
● Кукуруза консервированная 1 банка
● Сыр 200 г
● Лук репчатый 1 головка
● Чеснок 2 зубчика
● Ананас консервированный 1 банка
● Майонез по вкусу

Приготовление:

1. Обжарить грибы с луком с добавить немного соли. Старайтесь не резать грибы слишком мелко.
2. Отварить куриную грудку без кожи, нарезать кусочками и добавить к ней консервированную кукурузу.
3. Сыр также нарезать маленькими кубиками или натереть на крупной терке. Добавить к курице с кукурузой.
4. Смешать все ингредиенты с консервированным ананасом и заправить салат майонезом с чесноком.

Приятного аппетита!


1000

In [11]:
from random import seed, sample
#случайным образом выбираем тексты и смотрим, как они устроены
seed(40)
for text in sample(texts, 2):
    print(text)
    print('---' * 10)

Мексиканский суп с курицей

Нам потребуется:

Куриная грудка - 1 шт.,
Лук репчатый - 1 шт.,
Красная фасоль консервированная - 1 банка,
Вермишель - 50 г,
Кукуруза замороженная - 100 г,
Укроп - по вкусу,
Соль - по вкусу,
Молотый острый красный перец - по вкусу.

Из куриной грудки сварите бульон, достаньте мясо, отделите от костей и мелко нарежьте.

Лук мелко нашинкуйте. В глубоком сотейнике разогрейте растительное масло и обжарьте лук до прозрачности.

Добавьте к луку курицу и обжаривайте несколько минут. С фасоли слейте жидкость. Переложите фасоль в сотейник, добавьте молотый перец по вкусу, перемешайте и продолжайте обжаривать.

Залейте в сотейник куриный бульон, посолите по вкусу и варите 5 мин. Добавьте вермишель и кукурузу, варите еще 5 минут, затем накройте крышкой и снимите с огня.

Дайте супу настояться. Мелко нарубите укроп. Подавайте суп с укропом и кукурузными чипсами.
------------------------------
Мясной пирог

Ингредиенты:
для теста:
сливочное масло - 1/2 стак
сметана - 1 с

Эвристики: строки с ингредиентами короткие (пусть не больше 50 символов) и содержат числа 

In [0]:
import re

#функция, которая отбирает строки с ингредтиентами
def find_ingredient_sections(text):
    return re.findall(r'ингредиенты:(.+?)приготовление', text, re.I | re.S)

#проверяем длину строки и наличие числа
def maybe_ingredient(line):
    match = re.search(r'\d', line)
    size = len(line) <= 50
    return match and size

lines = []

#в переменную lines (список из строк) записываем только строки длиной меньше 50 символов, содержащих число
for text in texts:
    sections = find_ingredient_sections(text)
    for section in sections:
        for line in section.splitlines():
            if maybe_ingredient(line):
                lines.append(line)

In [14]:
print(lines[0])

● Филе куриной грудки 1 шт.


In [15]:
seed(14)
sample(lines, 20)

['● Оливковое масло - 2 ст. л.',
 'Молоко 1% - 100 г ',
 '- Морковь 2 - 3 шт.',
 'творог - 500 г,',
 '2 яйца,',
 '* 3/4 стакана холодной кипяченой воды',
 '• сыр - 100 г.',
 '1 нарезанный красный перец',
 '230 г кокосового или тростникового сахара ',
 'Яйцо - 3 шт.',
 'Сок лайма - 2 Ст. ложки',
 '● Мука - 100 г.',
 'Сметана - 50 г',
 '● Бананы - 3 шт.',
 'ананасы консервированные - 1 банка ',
 '● Сыр твердый 30 гр.',
 '500 мл растительного масла ',
 '5) соль, растительное масло для пассеровки овощей.',
 '5 средних варёных картофелин ',
 '● 3-5 зубчиков чеснока; ']

#Грамматика

In [0]:
from IPython.display import display

# pip install ipymarkup
from ipymarkup import show_markup, AsciiMarkup

from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_, dictionary,
    type, gram
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy.interpretation import fact, attribute


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans, AsciiMarkup)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
            display(facts)


INT = type('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
PRTF = gram('PRTF')
GENT = gram('gent')
DOT = eq('.')

TOKENIZER = MorphTokenizer()

##Spoon
В прошлый раз мы все строили постепенно, чтобы посмотреть, как работают отдельные фунции, предикаты и факты\
Теперь будет более системно строить парсер
(1) для начала исчислим все варианты меры "ложка"


In [18]:
Spoon = fact(
    'Spoon',
    ['type']
)


TEA = morph_pipeline([
    'ч',
    'чайн',
    'чайная',
]).interpretation(
    Spoon.type.const('чайная')
)

TABLE = morph_pipeline([
    'ст',
    'стол',
    'столовая',
]).interpretation(
    Spoon.type.const('столовая')
)

NAME = morph_pipeline([
    'л',
    'ложка'
])

SPOON = rule(
    or_(TEA, TABLE),
    DOT.optional(),
    NAME,
    DOT.optional()
).interpretation(
    Spoon
)


show_matches(
    SPOON,
    '2 Ст. ложки',
    '3 чайных л',
)

2 Ст. ложки
  ---------


Spoon(type='столовая')

3 чайных л
  --------


Spoon(type='чайная')